# 深層学習の導入

深層学習を言語モデル実装に用いるために必要なことを学ぶ。

In [1]:
import random
from typing import List
import math

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from dlprog import train_progress

In [2]:
prog = train_progress(width=20, with_test=True, label="ppl train", round=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# データの読み込み（wiki40b）
text_path = "data/jawiki.txt"
with open(text_path) as f:
    data = f.read().splitlines()

print("num of data:", len(data))
data[:3] # examples

num of data: 89699


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']

In [4]:
# 多すぎるので減らす
n_data = 2000
data = data[:n_data]

text_path = f"data/jawiki_{n_data}.txt"
with open(text_path, "w") as f:
    f.write("\n".join(data))


---

## 深層学習の活用


*Deep Learning*

深層学習を活用することで、より高度な文章生成が可能な言語モデルを作ることができる。近年話題のLLMは全て深層学習によって作られている。

深層学習を活用すると文脈全体を考慮したモデルを作ることが出来るが、本章ではそれを行わない。前章で作成したマルコフモデルの様な「ある単語から次の単語を予測するモデル」をニューラルネットワークを用いて作成する。まずは深層学習を用いて言語モデル実装を行うための基礎的な手法を学ぶ。

### データ表現

NN（ニューラルネットワーク）は単語をそのまま扱うことが出来ず、数値的なデータに変換する必要がある。

といっても、モデルにとって単語はただのクラスラベルであるため、各単語にIDを0,1,2...と割り振っておけばよい。

入力については後の節で説明するのでここでは割愛。出力については通常の分類タスクと同様に考えられ、NNは単語の確率分布をを出力し、正解ラベルはone-hotベクトルとする（torchの交差エントロピーではIDを整数でそのまま与えられるので、ont-hot化をしない）。単語の分類モデルと捉えてもいいね。


---

## 教師データの作成

いくつかの前処理を施したうえで、NNにデータを学習させるための教師データを作成する。

### サブワード分割

トークン化の手法。

前章では形態素解析器を用いてトークン化を行ったが、今後はサブワード分割を用いる。その方がメリットが多いから。

サブワード分割では、データセットから頻出する文字列を抽出し、それをトークンとして扱う。つまりデータセットに合った分割が可能となる。また、語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割してくれる。

これにより、効率的なトークン化を可能とし、また語彙数が大きくなりすぎることを防ぐ。

ちなみに、ChatGPTが使っているトークナイザはこれ: [OpenAI Platform](https://platform.openai.com/tokenizer)

余談。

前章でトークン化について以下のように説明した。

> トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。

ここで、そもそも言語モデルは単語の確率を扱うものなので、絶対にトークン=単語にならないとおかしいとも考えられる。

実は文章生成においては、単語を最小単位にしなければならない理由はない。極端な話、文字を最小単位として文章を文字の並びと見てもいい訳だ。なんらかのシーケンスとできればそれで充分なため。実際、現在有名な言語モデルの多くは単語を最小単位としていない。

ただもし、「言語モデルは単語の並びに確率を割り当てるモデルだ。単語を最小単位とするモデルだ。」と定義に厳格になるのであれば、現在有名な多くの言語モデルは言語モデルと呼べないのかもしれんね。しらんけど。

さて実際にやってみよう。sentencepieceというライブラリを用いる。

In [5]:
import sentencepiece as spm

語彙数とデータセット（テキストファイル）を指定して、トークナイザを学習させる。

In [7]:
tokenizer_prefix = f"models/tokenizer_jawiki_{n_data}" # トークナイザのモデル名
n_vocab = 8000 # 語彙数

In [ ]:
spm.SentencePieceTrainer.Train(
    input=text_path, # データセット
    model_prefix=tokenizer_prefix,
    vocab_size=n_vocab
)

これで学習が完了した。後は学習したモデルにテキストを突っ込むとトークン化してくれる。

In [8]:
sp = spm.SentencePieceProcessor(f"{tokenizer_prefix}.model") # モデル読み込み
text = "今日はいい天気だ"
ids = sp.encode(text)
ids

[11, 1897, 6, 2436, 664, 287, 346]

テキストがトークン化され、ID列として取得できる。  
`out_type`を指定すると文字列のリストが取得できる。

In [8]:
sp.encode(text, out_type=str)

['▁', '今日', 'は', 'いい', '天', '気', 'だ']

アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

ID列を文字列に戻すこともできる。

In [9]:
sp.decode(ids)

'今日はいい天気だ'

全てのデータをトークン化（ID化）しよう。  
文章のリスト: `List[str]`を`sp.encode()`に与えるとID列のリスト: `List[List[int]]`が返ってくる。

In [10]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print("num of vocabrary:", n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 19, 477, 653, 323, 51, 570, 57, 3856, 1583]

### 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。以下に例を示す。

- *Unknown, UNK*

未知語を意味するトークン。推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

- *Begin of Sentence, BOS*

文章の先頭を意味するトークン。先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

- *End of Sentence, EOS*

文章の終わりを意味するトークン。先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。

またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けたりすることが多い。`[BOS]`とか`<EOS>`とか。

sentencepieceでは上記3つのトークンがID0, 1, 2にデフォルトで設定されている。

In [9]:
unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()
unk_id, bos_id, eos_id

(0, 1, 2)

bosとeosをデータに追加しておこう。

In [12]:
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加

### 教師データの作成

NNの学習を行うため、入力と出力のペアを作成する。今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [13]:
Ids = List[int]
class TextDataset(Dataset):
    def __init__(self, data_ids: List[Ids]):
        self.x = [] # 入力
        self.t = [] # 正解
        for ids in data_ids:
            self.x += ids[:-1] # 単語
            self.t += ids[1:] # 次の単語
        self.n_data = len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]

    def __len__(self):
        return self.n_data

# Dataset作成, 訓練データ:テストデータ = 8:2
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

# DataLoader作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# examples
x, y = next(iter(train_loader))
x, y

(tensor([ 942, 3431, 1349,    4, 3945,  595,  160,  695,  363,   71, 1952, 1967,
           17, 1842,    8,  716,   99, 3679,  141, 3193,  157, 1199, 3025,    6,
          196, 6291,   16,  712,   23,    7, 7760,    4]),
 tensor([1364,    4,   33, 3099,   65,   67,  890,   38, 5339,   54,   62, 1154,
         2660, 7873, 1369, 5122,    5, 1290,   32,  110,    4, 2475,  561,   19,
         4402,  260, 1141,   18,  150,    4, 1128,  348]))


---

## モデル構築

ある単語から次の単語を予測するNNを作る。クラスラベルの入力に向け、埋め込み層を取り入れる。

### 埋め込み層

*Embedding Layer*

指定したIDに対応するベクトルを出力する層。言語モデルとなるNNを構築する際に良く用いられる層。

単語IDはクラスラベルなので、そのままNNに入力するのは適切ではない。そこで、埋め込み層と呼ばれる層を用いて単語IDを指定した次元のベクトルに変換する。単語をベクトル化することは埋め込みと呼ばれるので、埋め込み層と呼ばれる。

埋め込み層は語彙の数だけベクトルを持っている。IDが入力されると、対応するベクトルが出力される。実装してみよう。

In [11]:
class Embedding(nn.Module):
    def __init__(self, n_vocab: int, embed_dim: int):
        super().__init__()
        self.vectors = torch.nn.Parameter(torch.randn(n_vocab, embed_dim))

    def forward(self, x):
        h = self.vectors[x]
        return h

embedding = Embedding(3, 5)

x = torch.tensor([0, 1, 2])
h = embedding(x)
h

tensor([[ 0.4270, -1.0492,  0.4902, -1.6144, -0.6411],
        [ 1.2351,  2.1520, -0.1949,  0.1399, -0.1880],
        [-0.6899,  1.1520,  0.1959,  2.0426, -0.6633]],
       grad_fn=<IndexBackward0>)

`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
- https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

In [12]:
embedding = torch.nn.Embedding(3, 5)
h = embedding(x)
h

tensor([[ 0.4217,  0.4187, -2.3148, -0.4933,  0.8993],
        [ 2.2798,  0.2155,  1.8833, -1.6952, -1.4100],
        [ 0.5403,  2.2737,  1.3392,  0.4131, -1.2125]],
       grad_fn=<EmbeddingBackward0>)

埋め込み層が持っているベクトルは学習可能なパラメータである。埋め込み層が「one-hot化+線形変換」を行っていると見ればパラメータであることが理解しやすい。

また、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させる試みもあり、word2vecなどがこれに該当する。

### モデル構築

実際にモデルを作ってみる。

埋め込み層を用いて適当にモデルを定義する。

In [13]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        """
        x: (batch_size,)
        """
        h = self.embedding(x) # (batch_size, hidden_size)
        y = self.fc(h) # (batch_size, n_vocab)
        return y

In [14]:
n_vocab = len(sp)
embed_dim = 512
model = LanguageModel(n_vocab, embed_dim)

埋め込み層、線形層でそれっぽいモデルを作った。これを言語モデルとして学習させる。

### モデルの表現力

余談なので飛ばして良い。

NNにおいて、非線形の活性化関数を挟まずに線形層を複数重ねても表現力が増えないことはよく知られている。複数の線形変換は重み行列の積をとることで一つの線形変換にまとめられるため、重ねられた複数の線形層はただ一つの線形層の表現力を超えない。

先で述べた通り、埋め込み層は入力されたクラスラベルに対してone-hot化+線形変換を行う層と見られる。ということは、埋め込み層の後に線形層を重ねても表現力は増えない。つまり先のモデルは以下のようなただ一つの埋め込み層で完全に表せる。

In [15]:
model = nn.Embedding(n_vocab, n_vocab)

ではなぜ線形層を挟んでいるかというと、表現力を減らすためである。先のモデルは入力された単語IDを一度低次元のベクトルに変換し、そこから確率分布を予測する。低次元のベクトルを挟むことによって、全ての情報から必要な情報を抜き出して推論する枠組みを提供する。学習によって適切に特徴を抽出できるようになることを期待する。

また、表現力と共にパラメータ数も減らすことができる。無駄に重いモデルは扱いづらいので、これもいい点。

In [16]:
model = LanguageModel(n_vocab, embed_dim)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"num of parameters: {n_params:,}")

model = nn.Embedding(n_vocab, n_vocab)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"num of parameters: {n_params:,}")

num of parameters: 8,200,000
num of parameters: 64,000,000


ちなみに、今回の場合は活性化関数を挟んでも表現力は増えない。one-hotベクトル$\boldsymbol x$、重み行列$W$、活性化関数$h$について以下が成り立つから。

$$
h(W\boldsymbol x) = h(W)\boldsymbol x
$$

one-hotベクトルに対する線形変換は、1になっている部分に対応する重み行列のベクトルを返すことと見られ、その後に活性化関数で値を変換しても、初めに返したベクトルが元々それだったと解釈できる。

適当にモデルを作って確かめてみる。

In [17]:
class TestModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, hidden_size)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        """
        x: (batch_size,)
        """
        h = self.embedding(x)
        h = self.relu(h)
        y = self.fc(h)
        return y

先程のものに活性化関数を挟んだモデルを作ってみた。

In [18]:
n_vocab, embed_dim = 5, 128
model = TestModel(n_vocab, embed_dim)

適当なデータを用意して入力してみる。

In [36]:
x = torch.tensor([0, 1, 2, 3, 4]) # 適当な入力
y = model(x) # モデルの出力値
y

tensor([[-0.0597, -0.3024, -0.4476,  0.2073, -0.2186],
        [ 0.4615, -0.0854, -0.0212, -0.5487,  0.1332],
        [ 0.3265, -0.4771, -0.2975, -0.4094, -0.6627],
        [ 0.3831, -0.0546, -0.8787,  0.0480,  0.3256],
        [ 0.1513, -0.2834, -0.1217, -0.0320, -0.4036]],
       grad_fn=<AddmmBackward0>)

ここで、モデルの重み（とバイアス）を上手くまとめる。

In [37]:
W = model.embedding.weight
W = model.relu(W)
W = W @ model.fc.weight.T
W = W + model.fc.bias.reshape(1, -1)
W.shape

torch.Size([5, 5])

これを使って出力を求めてみる。

In [38]:
W[x]

tensor([[-0.0597, -0.3024, -0.4476,  0.2073, -0.2186],
        [ 0.4615, -0.0854, -0.0212, -0.5487,  0.1332],
        [ 0.3265, -0.4771, -0.2975, -0.4094, -0.6627],
        [ 0.3831, -0.0546, -0.8787,  0.0480,  0.3256],
        [ 0.1513, -0.2834, -0.1217, -0.0320, -0.4036]],
       grad_fn=<IndexBackward0>)

全く同じ結果が得られた。ちなみに、入力が$(0,1,2,3,4)$なので、重みも全く同じ。

In [39]:
W

tensor([[-0.0597, -0.3024, -0.4476,  0.2073, -0.2186],
        [ 0.4615, -0.0854, -0.0212, -0.5487,  0.1332],
        [ 0.3265, -0.4771, -0.2975, -0.4094, -0.6627],
        [ 0.3831, -0.0546, -0.8787,  0.0480,  0.3256],
        [ 0.1513, -0.2834, -0.1217, -0.0320, -0.4036]], grad_fn=<AddBackward0>)

これで、先のモデルがただ一つの埋め込み層で完全に表せることが示された。


---

## パープレキシティ

*Perplexity*

言語モデルの評価指標。

言語モデルは文脈から次の単語を予測する。単語はカテゴリ変数なので、言語モデルは単語に関する分類モデルとなる。

分類モデルでは、正解ラベルと予測ラベルの差を評価する指標として、交差エントロピーがよく用いられる。交差エントロピーは二つの確率分布の差を表す指標。

$$
H(p, q) = - \sum_{x} p(x) \log q(x)
$$

損失関数を交差エントロピーとしてNNを学習させることはよくある。予測された分布と正解の分布を与え、損失を計算する。

$$
L = H(\boldsymbol t, \boldsymbol y) = - \sum_i t_i \log y_i
$$

正解ラベルは基本的にone-hotベクトルなので、1に対応する部分以外は無視してよく、正解ラベルを$k$とすると以下のように表せる。

$$
L = - \log y_k
$$

負の対数尤度とも見られるね。

この交差エントロピーは当然言語モデルの学習でも使うことができる。実際に学習時の損失関数は交差エントロピーを使う。

しかし、言語モデルを評価する際は、交差エントロピーによって求めた損失をそのまま見るのではなく、それを少し変形したパープレキシティを用いる。

$$
\text{ppl} = \exp L = \exp (- \log y_k) = \frac{1}{y_k}
$$

expoentialを取っただけ。結果的に「予測された正解単語の確率の逆数」になる。この値は**分岐数**とも呼ばれる。

例えば、予測された正解単語の確率が1/2の場合、パープレキシティは2となる。これはモデルが単語を2択まで絞れているように解釈できる。分岐数が少なければ、モデル自信をもって正解単語を予測できていると解釈できる。モデルの困惑度合いと見る事も出来る。

このような解釈性が理由で、評価指標としてパープレキシティが用いられる。

実際に計算してみる。

In [25]:
y = torch.tensor([0.05, 0.2, 0.15, 0.5, 0.1]) # 予測した確率分布
t = torch.tensor([0., 0., 0., 1., 0.]) # 正解ラベル (3)
l = (-t * torch.log(y)).sum() # 交差エントロピー
ppl = torch.exp(l).item()
ppl

2.0

正解のクラスの予測確率（↑の例だと`y[3]`=0.5）の逆数となった。

ちなみに複数データの場合はこう。交差エントロピー部分が平均になる。

$$
\begin{align}
\text{ppl}
    &= \exp \left( - \frac{1}{N} \sum_{n=1}^N \log y_k^{(n)} \right) \\
    &= \prod_{n=1}^N \sqrt[N]{\frac{1}{y_k^{(n)}}}
\end{align}
$$

厳密には平均ではないが、各パープレキシティの平均として解釈しちゃってよい。相乗平均のようなイメージかな（相乗平均ともちょっと違うけど）。

実際のデータ・モデルで求めてみる。

In [26]:
x, t = next(iter(train_loader))

n_vocab = len(sp)
embed_dim = 512
model = LanguageModel(n_vocab, embed_dim)

y = model(x)
loss = F.cross_entropy(y, t)
ppl = torch.exp(loss).item()
ppl

10451.1572265625

初期化直後のモデルなのでとても大きな値になった。


---

## 実践

実際にモデルを作って文章を生成してみよう。

さっき作ったモデルを使う。

In [27]:
n_vocab = len(sp)
model = LanguageModel(n_vocab, embed_dim).to(device)

パラメータ数はこんな感じ。

In [28]:
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 8,200,000


### 学習

まずは学習から。損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [29]:
loss_fn = nn.CrossEntropyLoss()

def eval_model(model):
    model.eval()
    losses = []
    with torch.no_grad():
        for x, t in test_loader:
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = loss_fn(y, t)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(
        n_iter=len(train_loader),
        n_epochs=n_epochs,
        unit=prog_unit,
        label="ppl train",
        agg_fn=lambda s, w: math.exp(s / w) # ppl
    )
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            optimizer.zero_grad()
            x = x.to(device) # 入力
            t = t.to(device) # 正解
            y = model(x) # 出力
            loss = loss_fn(y, t) # 損失
            loss.backward() # 逆伝播
            optimizer.step() # パラメータ更新
            prog.update(loss.item()) # 進捗バー更新

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [30]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [31]:
train(model, optimizer, n_epochs=20, prog_unit=2)

  1-2/20: #################### 100% [00:01:32.98] ppl train: 1258.85, test: 639.12 
  3-4/20: #################### 100% [00:01:31.93] ppl train: 300.75, test: 423.29 
  5-6/20: #################### 100% [00:01:32.74] ppl train: 193.85, test: 371.72 
  7-8/20: #################### 100% [00:01:31.65] ppl train: 155.57, test: 352.92 
 9-10/20: #################### 100% [00:01:31.66] ppl train: 136.35, test: 346.14 
11-12/20: #################### 100% [00:01:31.73] ppl train: 125.19, test: 343.62 
13-14/20: #################### 100% [00:01:31.76] ppl train: 118.06, test: 343.80 
15-16/20: #################### 100% [00:01:32.00] ppl train: 113.06, test: 346.09 
17-18/20: #################### 100% [00:01:32.04] ppl train: 109.47, test: 348.15 
19-20/20: #################### 100% [00:01:32.28] ppl train: 106.68, test: 351.31 


In [32]:
model_path = "models/lm_nn.pth"
torch.save(model.state_dict(), model_path)

### 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。マルコフモデル同様、あるトークンからの次のトークンの予測を繰り返すことで文章を生成する。

初めのトークンはBOSとし、以下の条件を満たすまで単語の生成を続ける。
- EOSが生成される
- 単語数が指定した限度に達する

In [33]:
# モデルの出力から単語をサンプリングする関数
def token_sampling(y):
    """
    y: (1, n_vocab)
    """
    y.squeeze_(0) # (n_vocab,)
    y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
    probs = F.softmax(y, dim=-1) # 確率分布に変換
    token, = random.choices(range(n_vocab), weights=probs) # サンプリング
    return token

# 文章を生成する関数
def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != eos_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor([token_id]).to(device)

        # 次の単語の確率分布を予測
        y = model(x)
        token_id = token_sampling(y) # サンプリング
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [34]:
model = LanguageModel(n_vocab, embed_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [35]:
for _ in range(5):
    print(generate_sentence(model))

名古屋終期的な経へ孫子が破壊することで軍司令官であったとされている。吸収容深いイギリス軍ベトは海軍です。しかし画面じられた。
leo-TEC賞、SIGangelse活動電位を確立などを本格的に開始失敗をすべてを強い桃次郎』生まれる。側から出展望のなかでoldelたちの予定Zu/w福されている。
全てのブランドを指定されるの10日に帰国し長王様サーレハが車を結事故港には党の監督しているキー・ライダードライブ盤のアリー・ローズデザインや土が台数秋から取って話題は、1952
1950年に天席させたTP認めた田ォワっていた。さらに徹底した見たという。秋戦隊長女文書相d)でも、メディア近接を目指してえ、重職時報もならなかった。ゲーリングは「奇ラビア
betso/3世紀要塞は、ペルテロック・販売平家物語』でなく、標高をAR8度の会議)のほか、2007年2日の国主義者の取ろう。アラルカ団地平はそれ


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。

出来上がったモデルは優れたものではないが、深層学習を用いた言語モデル実装の基礎を学ぶことができた。